In [1]:
# !pip install pandas	
# !pip install numpy
# !pip install tqdm	
# !pip install matplotlib

In [1]:
# 라이브러리
import random
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy.random import randint, rand

In [2]:
# 데이터 불러오기
machine_info = pd.read_csv("./data/machine_info.csv")
order_info = pd.read_csv("./data/order_info.csv")
order_info_nl = pd.read_csv("./data/order_info_nl.csv", header=None)

In [3]:
order_info_nl.columns = ['영업납기', '중산도면', '단가', '수량', '선급', '자연어주문']

In [4]:
# 전처리 (1): column name 변경
order_info = order_info.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent'
})

machine_info = machine_info.rename(columns={
    'JSDWG': 'item',
    'MCNO': 'machine',
    'AVG_CT': 'capacity'
})

order_info_nl = order_info_nl.rename(columns={
    '영업납기': 'time',
    '중산도면': 'item',
    '단가': 'cost',
    '수량': 'qty',
    '선급': 'urgent',
    '자연어주문': 'nl_order'
})

In [5]:
# 전처리 - order
def preprocess_order(df):
    # 긴급 생산 요건이 있는 경우 1, 그렇지 않은 경우는 0
    df['urgent'] = df['urgent'].fillna(0)

    for i in range(len(df)):
        if df.loc[i, 'urgent'] != 0:
            df.loc[i, 'urgent'] = 1

    # 전처리 (3): time type 부여 (문자열 → datetime)
    df['time'] = df['time'].apply(
        lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')
    )

    df.dropna(axis=0, inplace=True)

    # 같은 종류의 order 묶기
    df = df.groupby(['time','item','cost','urgent']).sum().reset_index()
    
    return df

# 전처리 - machine
def preprocess_machine(df):
    df.dropna(axis=0, inplace=True)

    return df

In [7]:
# 슬라이딩 윈도우 함수 정의
def create_shifted_dataset(df, window_size=10):
    datasets = []
    for i in range(len(df) - window_size + 1):
        window = df.iloc[i:i+window_size].reset_index(drop=True)
        datasets.append(window)
    return datasets

In [7]:
machine_info = preprocess_machine(machine_info)

def make_dataset(order_info_nl, machine_info, window_size=10):
    dataset = create_shifted_dataset(order_info_nl, window_size=window_size)
    dataset_for_ga = list()
    dataset_for_nl = list()

    for data in dataset:
        order = data.iloc[:, :5]
        nl = data.iloc[:, 5:]

        order = preprocess_order(order)
        order = pd.merge(order, machine_info, on='item', how='inner')

        dataset_for_ga.append(order)
        dataset_for_nl.append(nl)

    return dataset_for_ga, dataset_for_nl

dataset_for_ga, dataset_for_nl = make_dataset(order_info_nl, machine_info)

In [8]:
def ga_preprocess(dataset):
    T = list(set(dataset['time']))
    I = list(set(dataset['item']))
    J = list(set(dataset['machine']))

    # cit = 날짜 t에, item i가 미생산 될 때 발생하는 비용
    cit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['cost']))[0]
                cit[i, t] = value
            else:
                cit[i, t] = 0

    # pit = 날짜 t에, item i가 긴급 생산이 필요한 경우 1, 그렇지 않으면 0
    pit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['urgent']))[0]
                pit[i, t] = value
            else:
                pit[i, t] = 0

    # dit = 날짜 t에 item i마다 생산되어야 하는 요구량
    dit = dict()
    for i in I:
        for t in T:
            temp_dataset = dataset[(dataset['item'] == i) & (dataset['time'] == t)]
            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['qty']))[0]
                dit[i, t] = value
            else:
                dit[i, t] = 0

    # mijt= 날짜 t에 NC machine j가 item i를 생산할 수 있는 능력
    mijt = dict()
    for i in I:
        for j in J:
            temp_dataset = dataset[
                (dataset['item'] == i) &
                (dataset['machine'] == j)
            ]

            if len(temp_dataset) != 0:
                value = list(set(temp_dataset['capacity']))[0]
                for t in T:
                    mijt[i, j, t] = value
            else:
                for t in T:
                    mijt[i, j, t] = 0

    return T, I, J, cit, pit, dit, mijt

# 머신 j가 시간 t에 아이템 i를 만들수 없을 경우 0으로 처리하기 위함함
def decode(mijt, xijt):
    for j in J:
        for t in T:
            for i in I:
                if mijt[i, j, t] == 0:
                    xijt[i, j, t] = 0
    return xijt

# 목적함수를 최적화하기 위한 xijt 조합을 임의로 생성하는 함수
def generation_xijt():
    xijt = {}

    for i in I:
        for j in J:
            for t in T:
                if dit[i, t] > 0:
                    xijt[i, j, t] = random.uniform(0, 1)
                else:
                    xijt[i, j, t] = 0

    xijt = decode(mijt, xijt)
    return xijt

def dict2list(xijt):
    return list(xijt.values())

def list2dict(bitstring, type='xijt'):
    if type == 'xijt':
        _keys = xijt_keys
    elif type == 'mijt':
        _keys = mijt_keys

    xijt = {}
    for idx, value in enumerate(bitstring):
        xijt[_keys[idx]] = value

    return xijt

# 제약조건 확인
# 기계 하루 총 생산시간은 600분
# j기계로 600분 안에 아이템 i를 생산할 수 없다면 아이템 생산하지 않도록 하기 위함.
def constraint_check(xijt):
    for j in J:
        for t in T:
            check_value = sum(mijt[i, j, t] * round(xijt[i, j, t] * dit[i, t]) for i in I) <= 600

            if check_value is False:
                for i in I:
                    xijt[i, j, t] = 0

    return xijt

# 목적 함수
def objective(xijt):
    uit = {}
    xijt = constraint_check(xijt)

    for i in I:
        for t in T:
            u = dit[i, t] - sum(round(xijt[i, j, t] * dit[i, t]) for j in J)
            if u >= 0:
                uit[i, t] = u
            else:
                uit[i, t] = abs(u) * 10_000_000

    objective = sum(uit[i, t] * cit[i, t] * pit[i, t] for i in I for t in T)
    return objective

# Tournament selection
def selection(pop, scores, n_pop):
    # n_pop은 한 세대를 구성하는 염색체 수이며, 하이퍼파라미터 값
    k = round(n_pop * 0.5)
    selection_ix = randint(len(pop))
    
    for ix in randint(0, len(pop), k - 1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
            
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    p1 = dict2list(p1)
    p2 = dict2list(p2)
    c1, c2 = p1.copy(), p2.copy()

    if rand() < r_cross:
        base_bitstring = dict2list(mijt)

        # crossover (1): 유효 인덱스 추출
        valid_index = []
        for i in range(len(base_bitstring)):
            if base_bitstring[i] > 0:
                valid_index.append(i)

        n = round(r_cross * len(valid_index))
        pt_0 = random.sample(valid_index, n)
        pt_1 = list(set(valid_index) - set(pt_0))

        # crossover (2): 교차 적용
        for i in pt_0:
            c1[i] = p2[i]
            c2[i] = p1[i]

        for i in pt_1:
            c1[i] = p2[i]
            c2[i] = p1[i]

    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if rand() < r_mut:
            base_bitstring = dict2list(mijt)

            # mutation (1): 유효 인덱스 찾기
            valid_index = []
            for j in range(len(base_bitstring)):
                if base_bitstring[j] > 0:
                    valid_index.append(j)

            n = round(r_mut * len(valid_index))

            # mutation (2): 무작위 인덱스에서 값 변경
            pt_0 = random.sample(valid_index, n)
            for idx in pt_0:
                bitstring[idx] = random.uniform(0, 1)

def genetic_algorithm(bounds, n_iter, n_pop, r_cross, r_mut):
    # Utility
    log = []
    log_detail = []
    best_gen = 0

    # GA algorithm 초기화
    pop = [generation_xijt() for _ in range(n_pop)]
    best = decode(mijt, pop[0])
    best_eval = objective(best)
    print(best_eval)

    for gen in tqdm(range(n_iter)):
        # 현재 세대 해를 디코딩
        decoded = [decode(mijt, p) for p in pop]

        # 목적함수 값 계산
        scores = [objective(d) for d in decoded]

        # 가장 좋은 해 갱신
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f'>best! {gen}, {scores[i]}')
                best_gen = gen
            else:
                # stop rule: 개선 없이 100세대 이상일 경우 중단
                if gen - best_gen > 100:
                    print('stop')
                    return [best, best_eval, log, log_detail]

        # 부모 해 집합 선택
        selected = [selection(pop, scores, n_pop) for _ in range(n_pop)]
        log_detail.append([gen, objective(selected[0])])

        # 자식 세대 생성
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i + 1]
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut)
                children.append(list2dict(c))

        pop = children
        log.append([gen, best_eval])

    return [best, best_eval, log, log_detail]

def set_hyper_parameters():
    # 하이퍼파라미터 설정
    hyper_parameters = pd.DataFrame({
        'index':    ['index_1', 'index_2', 'index_3', 'index_4',
                    'index_5', 'index_6', 'index_7', 'index_8'],
        'n_iter':   [200, 200, 200, 200, 200, 200, 200, 200],
        'n_pop':    [10, 20, 40, 20, 20, 20, 20, 20],
        'r_cross':  [0.4, 0.4, 0.4, 0.1, 0.2, 0.3, 0.4, 0.4],
        'r_mut':    [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.1, 0.6]
    })

    # 결과 기록용 열 추가
    hyper_parameters['objective'] = np.nan
    hyper_parameters['time'] = np.nan

    return hyper_parameters

def find_hyper_parameters(hyper_parameters):
    log_list = []

    for i in range(len(hyper_parameters)):
        parameter = hyper_parameters.iloc[i]
        index_nm = parameter['index']
        print(f'{index_nm}')

        start = datetime.datetime.now()

        # 초기 해 및 키 정의
        xijt = generation_xijt()
        global xijt_keys, mijt_keys
        xijt_keys = list(xijt.keys())
        mijt_keys = list(mijt.keys())

        # 하이퍼파라미터 설정
        n_iter = parameter['n_iter']
        n_pop = parameter['n_pop']
        r_cross = parameter['r_cross']
        r_mut = parameter['r_mut']

        # 유전 알고리즘 실행
        best, score, log, log_detail = genetic_algorithm(mijt, n_iter, n_pop, r_cross, r_mut)

        # 실행 시간 및 결과 기록
        elapsed_time = datetime.datetime.now() - start
        hyper_parameters.loc[i, 'time'] = elapsed_time
        hyper_parameters.loc[i, 'objective'] = score

        log_list.append(log)

    # 결과 출력
    return hyper_parameters, log_list


In [16]:
# # 적합한 하이퍼 파라미터 찾는 과정
# temp0 = [dataset_for_ga[i] for i in [0, 5, 10]]

# hyper_parameters_ls, log_list_ls = list(), list()

# # 유전 알고리즘
# def ga_get_gt(dataset_for_ga):
#     for data in dataset_for_ga:
#         global T, I, J, cit, pit, dit, mijt
#         T, I, J, cit, pit, dit, mijt = ga_preprocess(data)

#         hyper_parameters = set_hyper_parameters()
#         hyper_parameters, log_list = find_hyper_parameters(hyper_parameters)

#         hyper_parameters_ls.append(hyper_parameters)
#         log_list_ls.append(log_list)
#         print(hyper_parameters)

#     return hyper_parameters_ls, log_list_ls

# hyper_parameters_ls, log_list_ls = ga_get_gt(temp0)

In [15]:
import pickle

# 저장된 피클 불러오기기
# with open('log.pkl', 'rb') as f:
#     loaded_data = pickle.load(f)

# best_ls, score_ls, log_ls, log_detail_ls = loaded_data

# 데이터 설정정
# temp = [dataset_for_ga[i] for i in range(10)]

best_ls, score_ls, log_ls, log_detail_ls, gt_ls = list(), list(), list(), list(), list()

# 유전 알고리즘
def ga_get_gt(dataset_for_ga):
    data_len = len(dataset_for_ga)
    
    for i, data in enumerate(dataset_for_ga):
        global T, I, J, cit, pit, dit, mijt
        T, I, J, cit, pit, dit, mijt = ga_preprocess(data)

        # 입력 데이터 생성
        xijt = generation_xijt()
        global xijt_keys, mijt_keys
        xijt_keys = list(xijt.keys())
        mijt_keys = list(mijt.keys())

        # 유전 알고리즘 하이퍼파라미터 설정 - index 03 적용
        n_iter = 200       # 반복할 세대 수
        n_pop = 40         # 세대당 염색체 수
        r_cross = 0.4      # 교차율
        r_mut = 0.4        # 변이율

        # 유전 알고리즘 실행
        best, score, log, log_detail = genetic_algorithm(mijt, n_iter, n_pop, r_cross, r_mut)

        best_ls.append(best)
        score_ls.append(score)
        log_ls.append(log)
        log_detail_ls.append(log_detail)

        with open('log.pkl', 'wb') as f:
            pickle.dump([best_ls, score_ls, log_ls, log_detail_ls], f)
        print(f'{i+1}번째 log 저장 완료. 총 {data_len}개')

        # 해 솔루션 저장용 딕셔너리 초기화
        solution_ = dict()

        # xijt 디코딩
        xijt = decode(mijt, xijt)

        # best 해를 기반으로 해를 계산
        for i in I:
            for j in J:
                for t in T:
                    solution_[(i, j, t)] = round(best[i, j, t] * dit[i, t])

        # 딕셔너리를 DataFrame으로 변환
        sol = pd.DataFrame.from_dict(solution_, orient='index').reset_index()
        sol.columns = ['(item, machine, time)', 'qty']

        gt_ls.append(sol[sol['qty'] > 0])

        with open('gt.pkl', 'wb') as f:
            pickle.dump(gt_ls, f)

        # print('{}번째 gt 저장 완료. 총 {}개'.format(i + 1, data_len))

        # print('------------------------')
        # print()

ga_get_gt(dataset_for_ga)

35558309540310


  0%|          | 0/200 [00:00<?, ?it/s]

>best! 0, 28244100209389
>best! 0, 17868722929391


  0%|          | 1/200 [00:08<26:56,  8.12s/it]

>best! 1, 14986671860572
>best! 1, 10375394304005
>best! 1, 5842453039046
>best! 1, 4611298144006


...

>best! 10, 94454294360500


 46%|████▌     | 91/200 [08:55<10:49,  5.96s/it]

>best! 91, 76794754774290


 96%|█████████▌| 192/200 [18:45<00:46,  5.86s/it]

stop
118번째 log 저장 완료. 총 118개


In [10]:
def combine_input(data):
    combined_input = '\n'.join(f'• {text}' for text in data['nl_order'])

    return combined_input

# combine_input(dataset_for_nl[117])

In [57]:
import json

def combine_condition(data):
    data = data[['item', 'machine', 'capacity']].drop_duplicates()
    data['machine'] = data['machine'].apply(lambda x: str(int(x)).zfill(4))

    condition_json = data.to_dict(orient='records')  # list of dicts
    return json.dumps(condition_json, ensure_ascii=False, indent=2)  # 보기 좋게 출력

print(combine_condition(dataset_for_ga[117]))

[
  {
    "item": "069478",
    "machine": "0408",
    "capacity": 5.0
  },
  {
    "item": "061218",
    "machine": "0404",
    "capacity": 4.67
  },
  {
    "item": "061218",
    "machine": "0408",
    "capacity": 5.67
  },
  {
    "item": "061218",
    "machine": "0409",
    "capacity": 5.67
  },
  {
    "item": "061218",
    "machine": "0410",
    "capacity": 5.74
  },
  {
    "item": "061218",
    "machine": "0416",
    "capacity": 5.22
  },
  {
    "item": "061218",
    "machine": "0424",
    "capacity": 4.59
  },
  {
    "item": "061218",
    "machine": "0425",
    "capacity": 4.37
  },
  {
    "item": "061218",
    "machine": "0426",
    "capacity": 5.86
  },
  {
    "item": "061218",
    "machine": "0433",
    "capacity": 4.6
  },
  {
    "item": "061218",
    "machine": "0434",
    "capacity": 4.67
  },
  {
    "item": "058797",
    "machine": "0404",
    "capacity": 17.5
  },
  {
    "item": "058797",
    "machine": "0405",
    "capacity": 15.89
  },
  {
    "item": "058797"

In [58]:
def combine_output(data):
    data[['item', 'machine', 'time']] = pd.DataFrame(data['(item, machine, time)'].tolist(), index=data.index)

    data = data.drop(columns='(item, machine, time)')
    data = data[['item', 'machine', 'time', 'qty']]
    data['machine'] = data['machine'].apply(lambda x: str(int(x)).zfill(4))

    data['time'] = pd.to_datetime(data['time'])  # 보장
    data['time'] = data['time'].dt.strftime('%Y-%m-%d')  # 또는 원하는 포맷

    condition_json = data.to_dict(orient='records')  # list of dicts
    return json.dumps(condition_json, ensure_ascii=False, indent=2)  # 보기 좋게 출력

print(combine_output(gt_ls[117]))

[
  {
    "item": "069478",
    "machine": "0408",
    "time": "2021-07-16",
    "qty": 6
  },
  {
    "item": "K04037",
    "machine": "0401",
    "time": "2021-08-31",
    "qty": 19
  },
  {
    "item": "K04037",
    "machine": "0409",
    "time": "2021-08-31",
    "qty": 7
  },
  {
    "item": "K04037",
    "machine": "0424",
    "time": "2021-08-31",
    "qty": 19
  },
  {
    "item": "K04037",
    "machine": "0425",
    "time": "2021-08-31",
    "qty": 7
  },
  {
    "item": "K04037",
    "machine": "0426",
    "time": "2021-08-31",
    "qty": 74
  },
  {
    "item": "K04037",
    "machine": "0428",
    "time": "2021-08-31",
    "qty": 190
  },
  {
    "item": "058797",
    "machine": "0404",
    "time": "2021-07-23",
    "qty": 7
  },
  {
    "item": "058797",
    "machine": "0404",
    "time": "2021-07-26",
    "qty": 4
  },
  {
    "item": "058797",
    "machine": "0405",
    "time": "2021-07-23",
    "qty": 10
  },
  {
    "item": "058797",
    "machine": "0405",
    "time": "

In [59]:
dataset = list()

for idx, gt in enumerate(gt_ls):
    input = dataset_for_nl[idx]
    input = combine_input(input)

    condition = dataset_for_ga[idx]
    condition = combine_condition(condition)

    output = combine_output(gt)

    temp_dict = dict()
    temp_dict['input'] = input
    temp_dict['condition'] = condition
    temp_dict['output'] = output

    dataset.append(temp_dict)

In [60]:
temp = dataset[0]
temp

{'input': '• K04033 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 25,870원이야. 아 그리구 검사품\n• 헐 방금 K04031 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 16,229원, 검사품 이래\n• 저기 051718 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. 그리구 특별한 거 없음\n• 어우 빨리 봐봐. 056984 4개 21년 6월 3일까지 해야되는데 어떡하지? 단가는 36,533원이구 특별한 거 없음래\n• 057791 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 45,500원이야. 아 그리구 검사품\n• 이것좀 봐봐. K03948 85개 해달라는데 21년 6월 18일까지래. 57,641원, 검사품\n• 이것좀 봐봐. S00341 500개 해달라는데 21년 2월 10일까지래. 2,990원, 특별한 거 없음\n• 잠깐만 이거 확인좀, 052996 도면 2개 맞아? 21년 2월 18일까지구 20,000원에 하는거? 특별한 거 없음도 있네\n• 야 이거 급한거 같은데, 21년 3월 10일까지 S00341 500개 해야돼. 단가는 2,990원이구 특별한 거 없음래\n• 저기 S00341 도면 500개 해야돼. 21년 4월 10일까지구 2,990원. 그리구 특별한 거 없음',
 'condition': '[\n  {\n    "item": "S00341",\n    "machine": "0410",\n    "capacity": 4.6\n  },\n  {\n    "item": "052996",\n    "machine": "0410",\n    "capacity": 25.0\n  },\n  {\n    "item": "052996",\n    "machine": "0421",\n    "capacity": 200.0\n  },\n  {\n    "item": "K04033",\n    "machine": "0404",\n    "capacity": 12.84\n  },

In [61]:
print(temp['input'])

• K04033 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 25,870원이야. 아 그리구 검사품
• 헐 방금 K04031 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 16,229원, 검사품 이래
• 저기 051718 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. 그리구 특별한 거 없음
• 어우 빨리 봐봐. 056984 4개 21년 6월 3일까지 해야되는데 어떡하지? 단가는 36,533원이구 특별한 거 없음래
• 057791 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 45,500원이야. 아 그리구 검사품
• 이것좀 봐봐. K03948 85개 해달라는데 21년 6월 18일까지래. 57,641원, 검사품
• 이것좀 봐봐. S00341 500개 해달라는데 21년 2월 10일까지래. 2,990원, 특별한 거 없음
• 잠깐만 이거 확인좀, 052996 도면 2개 맞아? 21년 2월 18일까지구 20,000원에 하는거? 특별한 거 없음도 있네
• 야 이거 급한거 같은데, 21년 3월 10일까지 S00341 500개 해야돼. 단가는 2,990원이구 특별한 거 없음래
• 저기 S00341 도면 500개 해야돼. 21년 4월 10일까지구 2,990원. 그리구 특별한 거 없음


In [62]:
print(temp['condition'])

[
  {
    "item": "S00341",
    "machine": "0410",
    "capacity": 4.6
  },
  {
    "item": "052996",
    "machine": "0410",
    "capacity": 25.0
  },
  {
    "item": "052996",
    "machine": "0421",
    "capacity": 200.0
  },
  {
    "item": "K04033",
    "machine": "0404",
    "capacity": 12.84
  },
  {
    "item": "K04033",
    "machine": "0405",
    "capacity": 10.42
  },
  {
    "item": "K04033",
    "machine": "0407",
    "capacity": 8.69
  },
  {
    "item": "K04033",
    "machine": "0408",
    "capacity": 9.12
  },
  {
    "item": "K04033",
    "machine": "0409",
    "capacity": 10.33
  },
  {
    "item": "K04033",
    "machine": "0410",
    "capacity": 8.99
  },
  {
    "item": "K04033",
    "machine": "0412",
    "capacity": 12.32
  },
  {
    "item": "K04033",
    "machine": "0416",
    "capacity": 11.23
  },
  {
    "item": "K04033",
    "machine": "0422",
    "capacity": 4.0
  },
  {
    "item": "K04033",
    "machine": "0424",
    "capacity": 9.87
  },
  {
    "item": "K0

In [63]:
print(temp['output'])

[
  {
    "item": "K04033",
    "machine": "0405",
    "time": "2021-05-13",
    "qty": 14
  },
  {
    "item": "K04033",
    "machine": "0409",
    "time": "2021-05-13",
    "qty": 42
  },
  {
    "item": "K04033",
    "machine": "0422",
    "time": "2021-05-13",
    "qty": 4
  },
  {
    "item": "K04033",
    "machine": "0424",
    "time": "2021-05-13",
    "qty": 55
  },
  {
    "item": "K04033",
    "machine": "0433",
    "time": "2021-05-13",
    "qty": 49
  },
  {
    "item": "K04033",
    "machine": "0435",
    "time": "2021-05-13",
    "qty": 33
  },
  {
    "item": "K04033",
    "machine": "0436",
    "time": "2021-05-13",
    "qty": 26
  },
  {
    "item": "056984",
    "machine": "0408",
    "time": "2021-06-03",
    "qty": 3
  },
  {
    "item": "056984",
    "machine": "0409",
    "time": "2021-06-03",
    "qty": 1
  },
  {
    "item": "056984",
    "machine": "0410",
    "time": "2021-06-03",
    "qty": 3
  },
  {
    "item": "056984",
    "machine": "0424",
    "time": "

In [64]:
import pandas as pd

df = pd.DataFrame(dataset)  # 각 dict가 행(row)이 되고 key는 열(column)

df.head()

,input,condition,output
0,• K04033 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""K04033"",\n ""machine"": ..."
1,• 헐 방금 K04031 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
2,"• 저기 051718 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. ...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
3,• 어우 빨리 봐봐. 056984 4개 21년 6월 3일까지 해야되는데 어떡하지? ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."
4,• 057791 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."


In [65]:
df.to_csv('./dataset_118.csv', index=False)

In [12]:
# 데이터 형식 수정 2025-05-22

In [25]:
dataset = pd.read_csv("./dataset_118.csv")
dataset.head()

,input,condition,output
0,• K04033 맞지? 318개 하는거? 21년 5월 13일까지 하면 되구 단가가 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""K04033"",\n ""machine"": ..."
1,• 헐 방금 K04031 도면 왔는데 21년 5월 24일까지래. 383개 해야되구 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
2,"• 저기 051718 도면 19개 해야돼. 21년 5월 30일까지구 8,333원. ...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
3,• 어우 빨리 봐봐. 056984 4개 21년 6월 3일까지 해야되는데 어떡하지? ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."
4,• 057791 맞지? 196개 하는거? 21년 6월 18일까지 하면 되구 단가가 ...,"[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."


In [26]:
order_info_nl_eng = pd.read_csv("./data/order_info_nl_eng.csv")
order_info_nl_eng.columns = ['nl_order']
order_info_nl_eng.head()

,nl_order
0,"K04033, right? 318? You can do it by May 13th,..."
1,"Oh, I just got the K04031 drawing, and it says..."
2,"There, I need to do 19 drawings for 051718. It..."
3,"Hey, look quickly. 056984 4 I have to do it by..."
4,"057791, right? Doing 196? You can do it until ..."


In [27]:
inputs = create_shifted_dataset(order_info_nl_eng, window_size=10)
input
inputs[0]

,nl_order
0,"K04033, right? 318? You can do it by May 13th,..."
1,"Oh, I just got the K04031 drawing, and it says..."
2,"There, I need to do 19 drawings for 051718. It..."
3,"Hey, look quickly. 056984 4 I have to do it by..."
4,"057791, right? Doing 196? You can do it until ..."
5,"Look at this. They want 85 K03948s, but they s..."
6,"Check this out. S00341 500 units, until Februa..."
7,"Wait, check this. Are these 2 drawings for 052..."
8,"Hey, this seems urgent. I need to get 500 S003..."
9,"There, I need to do 500 S00341 drawings. It's ..."


In [28]:
input_ls = list()
for input in inputs:
    input = combine_input(input)
    input_ls.append(input)

input_ls

["• K04033, right? 318? You can do it by May 13th, 2021, and the unit price is 25,870 won. Oh, and the inspection product.\n• Oh, I just got the K04031 drawing, and it says it's due on May 24th, 2021. I need to do 383 of them, and it costs 16,229 won, and it's an inspection item.\n• There, I need to do 19 drawings for 051718. It's until May 30th, 2021. It's 8,333 won. And nothing special.\n• Hey, look quickly. 056984 4 I have to do it by June 3rd, 2021. What should I do? The unit price is 36,533 won and there's nothing special about it.\n• 057791, right? Doing 196? You can do it until June 18th, 2021, and the unit price is 45,500 won. Oh, and the inspection product\n• Look at this. They want 85 K03948s, but they say it's due by June 18, 2021. 57,641 won, inspection item.\n• Check this out. S00341 500 units, until February 10th, 2021. 2,990 won, nothing special.\n• Wait, check this. Are these 2 drawings for 052996? It's until February 18th, 2021, and it's 20,000 won? There's nothing spe

In [29]:
dataset[['input']] = np.array(input_ls).reshape(-1, 1)
dataset.head()

,input,condition,output
0,"• K04033, right? 318? You can do it by May 13t...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""K04033"",\n ""machine"": ..."
1,"• Oh, I just got the K04031 drawing, and it sa...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
2,"• There, I need to do 19 drawings for 051718. ...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
3,"• Hey, look quickly. 056984 4 I have to do it ...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."
4,"• 057791, right? Doing 196? You can do it unti...","[\n {\n ""item"": ""S00341"",\n ""machine"": ...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."


In [30]:
import json

def parse_condition(json_text):
    """
    JSON 형식의 텍스트를 list of dicts 형태로 파싱
    """
    return json.loads(json_text)


In [31]:
for idx in range(len(dataset)):
    temp = parse_condition(dataset[['condition']].iloc[idx, :].condition)
    temp = "\n".join(
        f"• item: {d['item']}, machine: {d['machine']}, capacity: {d['capacity']}"
        for d in temp)

    dataset.iloc[idx, 1] = temp

dataset.head()

,input,condition,output
0,"• K04033, right? 318? You can do it by May 13t...","• item: S00341, machine: 0410, capacity: 4.6\n...","[\n {\n ""item"": ""K04033"",\n ""machine"": ..."
1,"• Oh, I just got the K04031 drawing, and it sa...","• item: S00341, machine: 0410, capacity: 4.6\n...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
2,"• There, I need to do 19 drawings for 051718. ...","• item: S00341, machine: 0410, capacity: 4.6\n...","[\n {\n ""item"": ""056984"",\n ""machine"": ..."
3,"• Hey, look quickly. 056984 4 I have to do it ...","• item: S00341, machine: 0410, capacity: 4.6\n...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."
4,"• 057791, right? Doing 196? You can do it unti...","• item: S00341, machine: 0410, capacity: 4.6\n...","[\n {\n ""item"": ""057791"",\n ""machine"": ..."


In [32]:
dataset[['condition']].iloc[0, :]

condition    • item: S00341, machine: 0410, capacity: 4.6\n...
Name: 0, dtype: object

In [33]:
print(dataset[['condition']].iloc[0, :].condition)

• item: S00341, machine: 0410, capacity: 4.6
• item: 052996, machine: 0410, capacity: 25.0
• item: 052996, machine: 0421, capacity: 200.0
• item: K04033, machine: 0404, capacity: 12.84
• item: K04033, machine: 0405, capacity: 10.42
• item: K04033, machine: 0407, capacity: 8.69
• item: K04033, machine: 0408, capacity: 9.12
• item: K04033, machine: 0409, capacity: 10.33
• item: K04033, machine: 0410, capacity: 8.99
• item: K04033, machine: 0412, capacity: 12.32
• item: K04033, machine: 0416, capacity: 11.23
• item: K04033, machine: 0422, capacity: 4.0
• item: K04033, machine: 0424, capacity: 9.87
• item: K04033, machine: 0426, capacity: 11.77
• item: K04033, machine: 0433, capacity: 9.54
• item: K04033, machine: 0434, capacity: 13.05
• item: K04033, machine: 0435, capacity: 9.0
• item: K04033, machine: 0436, capacity: 9.0
• item: K04033, machine: 0438, capacity: 4.4
• item: K04033, machine: 0439, capacity: 12.47
• item: K04033, machine: 0440, capacity: 9.4
• item: K04031, machine: 0407, 

In [34]:
parse_condition(dataset[['output']].iloc[0, :].output)

[{'item': 'K04033', 'machine': '0405', 'time': '2021-05-13', 'qty': 14},
 {'item': 'K04033', 'machine': '0409', 'time': '2021-05-13', 'qty': 42},
 {'item': 'K04033', 'machine': '0422', 'time': '2021-05-13', 'qty': 4},
 {'item': 'K04033', 'machine': '0424', 'time': '2021-05-13', 'qty': 55},
 {'item': 'K04033', 'machine': '0433', 'time': '2021-05-13', 'qty': 49},
 {'item': 'K04033', 'machine': '0435', 'time': '2021-05-13', 'qty': 33},
 {'item': 'K04033', 'machine': '0436', 'time': '2021-05-13', 'qty': 26},
 {'item': '056984', 'machine': '0408', 'time': '2021-06-03', 'qty': 3},
 {'item': '056984', 'machine': '0409', 'time': '2021-06-03', 'qty': 1},
 {'item': '056984', 'machine': '0410', 'time': '2021-06-03', 'qty': 3},
 {'item': '056984', 'machine': '0424', 'time': '2021-06-03', 'qty': 4},
 {'item': '056984', 'machine': '0434', 'time': '2021-06-03', 'qty': 3},
 {'item': '056984', 'machine': '0439', 'time': '2021-06-03', 'qty': 2},
 {'item': '056984', 'machine': '0440', 'time': '2021-06-03

In [35]:
for idx in range(len(dataset)):
    temp = parse_condition(dataset[['output']].iloc[idx, :].output)
    temp = "\n".join(
        f"• item: {d['item']}, machine: {d['machine']}, time: {d['time']}, quantity: {d['qty']}"
        for d in temp)

    dataset.iloc[idx, 2] = temp

dataset.head()

,input,condition,output
0,"• K04033, right? 318? You can do it by May 13t...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: K04033, machine: 0405, time: 2021-05-1..."
1,"• Oh, I just got the K04031 drawing, and it sa...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0..."
2,"• There, I need to do 19 drawings for 051718. ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0..."
3,"• Hey, look quickly. 056984 4 I have to do it ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0424, time: 2021-06-1..."
4,"• 057791, right? Doing 196? You can do it unti...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0403, time: 2021-06-1..."


In [20]:
print(dataset[['output']].iloc[0, :].output)

• item: K04033, machine: 0405, time: 2021-05-13, quantity: 14
• item: K04033, machine: 0409, time: 2021-05-13, quantity: 42
• item: K04033, machine: 0422, time: 2021-05-13, quantity: 4
• item: K04033, machine: 0424, time: 2021-05-13, quantity: 55
• item: K04033, machine: 0433, time: 2021-05-13, quantity: 49
• item: K04033, machine: 0435, time: 2021-05-13, quantity: 33
• item: K04033, machine: 0436, time: 2021-05-13, quantity: 26
• item: 056984, machine: 0408, time: 2021-06-03, quantity: 3
• item: 056984, machine: 0409, time: 2021-06-03, quantity: 1
• item: 056984, machine: 0410, time: 2021-06-03, quantity: 3
• item: 056984, machine: 0424, time: 2021-06-03, quantity: 4
• item: 056984, machine: 0434, time: 2021-06-03, quantity: 3
• item: 056984, machine: 0439, time: 2021-06-03, quantity: 2
• item: 056984, machine: 0440, time: 2021-06-03, quantity: 1
• item: 057791, machine: 0421, time: 2021-06-18, quantity: 26
• item: 057791, machine: 0437, time: 2021-06-18, quantity: 72
• item: 057791, 

In [21]:
dataset['conditioning'] = (
    "Order information\n" + dataset['input'] + "\n\n" +
    "machine information\n" + dataset['condition']
)

In [22]:
print(dataset['conditioning'][0])

Order information
• K04033, right? 318? You can do it by May 13th, 2021, and the unit price is 25,870 won. Oh, and the inspection product.
• Oh, I just got the K04031 drawing, and it says it's due on May 24th, 2021. I need to do 383 of them, and it costs 16,229 won, and it's an inspection item.
• There, I need to do 19 drawings for 051718. It's until May 30th, 2021. It's 8,333 won. And nothing special.
• Hey, look quickly. 056984 4 I have to do it by June 3rd, 2021. What should I do? The unit price is 36,533 won and there's nothing special about it.
• 057791, right? Doing 196? You can do it until June 18th, 2021, and the unit price is 45,500 won. Oh, and the inspection product
• Look at this. They want 85 K03948s, but they say it's due by June 18, 2021. 57,641 won, inspection item.
• Check this out. S00341 500 units, until February 10th, 2021. 2,990 won, nothing special.
• Wait, check this. Are these 2 drawings for 052996? It's until February 18th, 2021, and it's 20,000 won? There's no

In [23]:
dataset.head()

,input,condition,output,conditioning
0,"• K04033, right? 318? You can do it by May 13t...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: K04033, machine: 0405, time: 2021-05-1...","Order information\n• K04033, right? 318? You c..."
1,"• Oh, I just got the K04031 drawing, and it sa...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0...","Order information\n• Oh, I just got the K04031..."
2,"• There, I need to do 19 drawings for 051718. ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0...","Order information\n• There, I need to do 19 dr..."
3,"• Hey, look quickly. 056984 4 I have to do it ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0424, time: 2021-06-1...","Order information\n• Hey, look quickly. 056984..."
4,"• 057791, right? Doing 196? You can do it unti...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0403, time: 2021-06-1...","Order information\n• 057791, right? Doing 196?..."


In [24]:
dataset = dataset.drop(['input', 'condition'], axis=1)[['conditioning', 'output']]

In [76]:
dataset.to_csv('./dataset_118_eng.csv', index=False)

In [38]:
# 상선님 요청 데이터

dataset.to_csv('./dataset_118_eng_for상선.csv', index=False)
dataset.head()

,input,condition,output
0,"• K04033, right? 318? You can do it by May 13t...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: K04033, machine: 0405, time: 2021-05-1..."
1,"• Oh, I just got the K04031 drawing, and it sa...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0..."
2,"• There, I need to do 19 drawings for 051718. ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 056984, machine: 0408, time: 2021-06-0..."
3,"• Hey, look quickly. 056984 4 I have to do it ...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0424, time: 2021-06-1..."
4,"• 057791, right? Doing 196? You can do it unti...","• item: S00341, machine: 0410, capacity: 4.6\n...","• item: 057791, machine: 0403, time: 2021-06-1..."
